In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from IPython.display import display
%reload_ext autoreload
%autoreload 2

import shutil
usetex = True if shutil.which('latex') else False
print("found latex:", usetex)
plt.rcParams.update({
    "text.usetex": usetex,
    "font.family": "DejaVu Sans",
    "font.size": 12,
})
plt.rc('text.latex', preamble=r'\usepackage{bm}')
figsize = 7

#%matplotlib notebook
%matplotlib inline

from lifters.plotting_tools import *

#df = pd.read_pickle("../_results/study_stereo1d_zero_noise.pkl")
import sys
sys.path.append("../_scripts/")
from noise_study import NOISE_DICT

# Constraints study

In [ ]:
df = pd.read_pickle("../_results/constraints_big_stereo2d_urT_learned.pkl")
df

In [ ]:
markers = {"all": ".", "bnb": "x"}
shuffle_names = {-1: "by $\\lambda$ value", 0:"in basis order"}
shuffle_names.update({i: f"random {i}" for i in range(1, 10)})
for seed, df_sub in df.groupby("random_noise_seed"):
    fig, ax = plt.subplots()
    fig.set_size_inches(7, 4)
    ax.set_title(f"noise seed {seed}")
    for (method, shuffle), df_row in df_sub.groupby(["method","shuffle_seed"]):
        if method == "bnb":
            continue
            
        color = f"C{shuffle+1}"
        assert len(df_row) == 1
        row = df_row.iloc[0]
        ax.scatter(row.costs.keys(), 
                    row.costs.values(),
                    color=color, 
                    marker=markers[method],
                    label=shuffle_names[shuffle])
        ax.axvline(row.num_constraints, 
                    ls=":", 
                    color=color) 
        ax.axhline(row.dual_cost, ls=":", color=color)
        
        if (shuffle == -1) and (method == "all"):
            for lamdas in [np.abs(row.lamda), np.cumsum(np.abs(row.lamda))]:
                fig2, ax2 = plt.subplots()
                fig2.set_size_inches(7, 2)
                ax2.scatter(range(len(lamdas)), lamdas, color=color)
                ax2.axvline(row.num_constraints, ls=":", color=color)
                ax2.set_ylabel("$\\lambda$ value")
                ax2.set_xlabel("constraint number")
                ax2.set_xlim(0, len(row.costs))
                ax2.grid()
    ax.axhline(row.qcqp_cost, ls="-", color="k", label="qcqp cost")
    ax.set_yscale("log")
    ax.set_xlabel("constraint number")
    ax.set_ylabel("dual / qcqp cost")
    ax.legend(title="ordering")
    ax.set_xlim(0, len(row.costs))
    ax.grid()

In [ ]:
# pd.reset_option("display.max_colwidth")
# pd.set_option("display.max_rows", 50)
df = pd.read_pickle("../_results/constraints_big_stereo2d_urT_details_known.pkl")
#df = pd.read_pickle("../_results/constraints_big_stereo2d_urT_details_learned.pkl")

for label in ["l dual", "l optimization"]:
    df.loc[:, label] = np.abs(df[label])
df

In [ ]:
import seaborn as sns
fig, ax = plt.subplots()
ax.scatter(range(len(df)),sorted(df["l optimization"])[::-1], label="optimization")
ax.scatter(range(len(df)),sorted(df["l dual"])[::-1], label="dual")
ax.set_yscale("log")
ax.legend()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=df, x="l dual", y="l optimization", hue="sparsity", palette="inferno")
ax.set_xscale('log')
ax.set_yscale('log')

fig, ax = plt.subplots()
sns.scatterplot(data=df, x="l dual", y="singular values", hue="sparsity", palette="inferno")
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
from IPython.display import display, Math, Latex
df.sort_values(by="l optimization", inplace=True, ascending=False)
df_small = df.iloc[:30].reset_index()
display(df_small)
for i, row in df.iloc[:30].iterrows():
    display(Latex(row.expression))

# Noise study

In [ ]:
# just one noise

import itertools

d_list = [2, 3]

noise = 0
lifter_types = ["slam1", "slam2"]
appendix = ""#"_sparse"
for d, lifter_type in itertools.product(d_list, lifter_types):
        
    fig, ax = plt.subplots()
    fig.set_size_inches(5, 3)
    fig.suptitle(f"{d}d range {lifter_type}", y=1.0)
    name = f"study_range{lifter_type}_{d}d_{noise}noise{appendix}"
    try:
        fname = f"../_results/{name}.pkl"
        df = pd.read_pickle(fname)
    except:
        print("didn't find", fname)
        print(f"no results yet for {name} (looked at {fname})")
        continue

    if len(df) == 0:
        print(f"no data for {name}")
        continue
    df["dual cost"] = df["dual cost"].abs()

    plot_tightness(df, ax=ax)
    ax.legend([])
    ax.set_title(f"noise {NOISE_DICT[noise]:.0g}")
    ax.set_xlabel("constraints added")
    savefig(fig, f"../_plots/study_range{lifter_type}_{d}d_noise{appendix}.png")

In [ ]:
import itertools

d_list = [2, 3]
#d_list = [3]
noise_list = list(NOISE_DICT.keys())[:5]
lifter_types = ["slam1"]
for d, lifter_type in itertools.product(d_list, lifter_types):
        
    fig, axs = plt.subplots(1, len(noise_list), sharey=True, sharex=True, squeeze=False)
    axs = axs.flatten()
    fig.set_size_inches(len(noise_list)*3, 3)
    fig.suptitle(f"{d}d range {lifter_type}", y=1.0)
    for ax, noise in zip(axs, noise_list):
    
        name = f"study_range{lifter_type}_{d}d_{noise}noise"
        try:
            fname = f"../_results/{name}.pkl"
            df = pd.read_pickle(fname)
        except:
            print("didn't find", fname)
            print(f"no results yet for {name}")
            continue
            
        if len(df) == 0:
            print(f"no data for {name}")
            continue

        plot_tightness(df, ax=ax)
        ax.legend([])
        ax.set_title(f"noise {NOISE_DICT[noise]:.0g}")
        ax.set_xlabel("constraints added")
    savefig(fig, f"../_plots/study_range{lifter_type}_{d}d_noise.png")

In [ ]:
#df = pd.read_pickle("../_results/study_stereo1d_zero_noise.pkl")
import sys
sys.path.append("../_scripts/")
from noise_study import NOISE_DICT

import itertools

d_list = [1, 2, 3]
#d_list = [3]
noise_list = list(NOISE_DICT.keys())[:5]
level_list = ["no", "urT"]
for d, level in itertools.product(d_list, level_list):
    if d in [1] and level == "urT":
        continue
        
    fig, axs = plt.subplots(1, len(noise_list), sharey=True, sharex=True, squeeze=False)
    axs = axs.flatten()
    fig.set_size_inches(len(noise_list)*3, 3)
    fig.suptitle(f"{d}d stereo, Lassere level {level}", y=1.0)
    for ax, noise in zip(axs, noise_list):
    
        name = f"study_stereo{d}d_{noise}noise_{level}level"
        try:
            fname = f"../_results/{name}.pkl"
            df = pd.read_pickle(fname)
            print("read", fname)
        except:
            print("didn't find", fname)
            print(f"no results yet for {name}")
            continue
            
        if len(df) == 0:
            print(f"no data for {name}")
            continue

        plot_tightness(df, ax=ax)
        ax.legend([])
        ax.set_title(f"noise {NOISE_DICT[noise]:.0g}")
        ax.set_xlabel("constraints added")
    
    
    savefig(fig, f"../_plots/study_stereo{d}d_noise_{level}.png")

# Scaling study

In [ ]:
def poly_label(coeffs, xlabel="x"):
    label = ""
    for i, c in enumerate(coeffs):
        deg = len(coeffs) - i
        if deg > 1:
            if abs(c) > 1e-10:
                label += f"{c:.1f}${xlabel}^{deg}$ + "
        elif deg == 1:
            if abs(c) > 1e-10:
                label += f"{c:.1f}${xlabel}$ + "
        else:
            label += f"{c:.1f}"
    return label

In [ ]:
from scipy.optimize import curve_fit
from lifters.plotting_tools import savefig

plot_labels = ["variables", "redundant constraints", "moment constraints", "total constraints"]
groupby = ["d"]
x_name = "landmarks"
x_label = "K"

for name, level in zip(["stereo_study", "stereo_study_lasserre"],
                       ["no", "urT"]):
    df = pd.read_pickle(f"../_results/{name}.pkl") # added extra constraints

    df["total constraints"] = df["found"]
    df["moment constraints"] = df["substitutions"]
    df["redundant constraints"] = df["total constraints"] - df["moment constraints"]
    df.drop(["found", "substitutions", "known"], axis=1, inplace=True)

    #display(df)
    
    for group, df_group in df.groupby(groupby):
        #display(df_group)
        x = df_group[x_name].unique()

        fig, ax = plt.subplots()
        for i, label in enumerate(plot_labels):
            y = np.r_[df_group[label]]
            ax.plot(x, y, label=label, color=f"C{i+1}")

            coeffs = np.polyfit(x, y, 2)
            #coeffs = np.round(coeffs, 0)
            poly = np.poly1d(coeffs)
            label= poly_label(poly, xlabel=x_label) #f"{coeffs[0]:.1f}$x^2$ + {coeffs[1]:.1f}$x$ + {coeffs[2]:.1f}"
            ax.plot(x, poly(x), label=label, ls="", marker='o', color=f"C{i+1}")
        ax.legend(loc="upper left")
        ax.set_xlabel(f"{x_name} ${x_label}$")
        ax.set_xticks(x)
        dim=','.join([f"{v}{k}" for k, v in zip(groupby, [group])])
        title=f"{dim} with {level} Lasserre"
        ax.set_title(title)
        ax.grid()
        savefig(fig, f"../_plots/study_stereo{dim}_{level}.png")

# Next experiments